# Creacion Consolidado por Materia Penal

Comenzaremos el analisis de la data consolidando la info en un unico DataSet, para ello vamos a considerar los siguientes puntos:


- DATA:  Se considerará toda las causas que ingresaron al Sistema desde 2015 al 2019, esto ya que se cuenta además con la data de las audiencias en ese período.


- No se consideran causas anteriores a este período.

- Se considerarán Causas sin termino.

In [1]:
import pandas as pd
import numpy as np
import os
import warnings

from tqdm import tqdm
from unicodedata import normalize
from pjud.data import cleandata
from pjud.data import transformdata

In [2]:
warnings.filterwarnings(action="ignore")
tqdm.pandas()

In [3]:
pd.set_option('display.max_columns', 100)

In [4]:
path_processed = "../data/processed/pjud"

#### Paso 1 : Consolidacion de Dataset x Materia y x rol

In [5]:
df_ingresos_materia = pd.read_feather(f"{path_processed}/processes_IngresosMateria.feather")
print(f"{len(df_ingresos_materia)} causas en el dataset de ingresos por materia")

3210643 causas en el dataset de ingresos por materia


In [6]:
df_termino_materia = pd.read_feather(f"{path_processed}/processes_TerminosMateria.feather")
print(f"Existen : {len(df_termino_materia)} causas en el dataset de termino por materia")

Existen : 3141268 causas en el dataset de termino por materia


Analizamos la data, uniendo ambos dataset.

In [7]:
df_fulldata_materia = pd.merge(df_ingresos_materia, df_termino_materia, how='outer', on=['COD. TRIBUNAL','RIT','COD. MATERIA'])
print(f"Existen: {len(df_fulldata_materia)} causas en total uniendo ambos dataset, aca existen datos sin registro de ingreso")

Existen: 3653836 causas en total uniendo ambos dataset, aca existen datos sin registro de ingreso


In [8]:
df_fulldata_materia['AÑO INGRESO'].value_counts()

2015.0    708906
2016.0    702869
2018.0    701128
2017.0    694117
2019.0    686373
Name: AÑO INGRESO, dtype: int64

### Analisis de la Data Causas por Materia

De estos dos dataset debo considerar:

- Caso 1: el dataset df_causas_terminadas completo
- Caso 2: Debo agregar además las causas que tengan Fecha Ingreso_x pero que aún no terminan.
- Caso 3: Debo analizar las causas que no tienen Fecha Ingreso_x pero si tienen Fecha Ingreso_y y Fecha Ingreso_y, ya que deben ser las causas que no fueron tomadas por la consulta de Ingresos. Acá se debe reemplazar los datos obtenidos en Termino Materia a Ingresos.

Luego de tener estos tres dataset debo concatenarlos

In [9]:
df_fulldata_materia

,index_x,COD. CORTE_x,CORTE_x,COD. TRIBUNAL,TRIBUNAL_x,RIT,TIPO CAUSA_x,COD. MATERIA,MATERIA_x,FECHA INGRESO_x,MES INGRESO,AÑO INGRESO,TOTAL INGRESOS POR MATERIAS,index_y,COD. CORTE_y,CORTE_y,TRIBUNAL_y,TIPO CAUSA_y,MATERIA_y,FECHA INGRESO_y,FECHA TERMINO,MES TERMINO,AÑO TERMINO,MOTIVO TERMINO,DURACION CAUSA,TOTAL TERMINOS
0,0.0,10.0,C.A. DE ARICA,988,TRIBUNAL DE JUICIO ORAL EN LO PENAL ARICA,100-2018,Ordinaria,7007,TRAFICO ILICITO DE DROGAS (ART. 3).,2018-04-02,abr-18,2018.0,1.0,0.0,10.0,C.A. DE ARICA,TRIBUNAL DE JUICIO ORAL EN LO PENAL ARICA,Ordinaria,TRAFICO ILICITO DE DROGAS (ART. 3).,2018-04-02,2018-06-01,jun-18,2018.0,SENTENCIA,60.0,1.0
1,1.0,10.0,C.A. DE ARICA,988,TRIBUNAL DE JUICIO ORAL EN LO PENAL ARICA,100-2018,Ordinaria,10004,"PORTE ILEGAL DE ARMA DE FUEGO, MUNICIONES Y OT...",2018-04-02,abr-18,2018.0,1.0,1.0,10.0,C.A. DE ARICA,TRIBUNAL DE JUICIO ORAL EN LO PENAL ARICA,Ordinaria,"PORTE ILEGAL DE ARMA DE FUEGO, MUNICIONES Y OT...",2018-04-02,2018-06-01,jun-18,2018.0,SENTENCIA,60.0,1.0
2,2.0,10.0,C.A. DE ARICA,988,TRIBUNAL DE JUICIO ORAL EN LO PENAL ARICA,101-2018,Ordinaria,12149,DESACATO (ART. 240 CODIGO DE PROCEDIMIENTO CIV...,2018-04-02,abr-18,2018.0,1.0,2.0,10.0,C.A. DE ARICA,TRIBUNAL DE JUICIO ORAL EN LO PENAL ARICA,Ordinaria,DESACATO (ART. 240 CODIGO DE PROCEDIMIENTO CIV...,2018-04-02,2018-08-29,ago-18,2018.0,SENTENCIA,149.0,1.0
3,3.0,10.0,C.A. DE ARICA,988,TRIBUNAL DE JUICIO ORAL EN LO PENAL ARICA,10-2018,Ordinaria,7037,TRAFICO DE PEQUEÑAS CANTIDADES (ART. 4).,2018-01-10,ene-18,2018.0,1.0,3.0,10.0,C.A. DE ARICA,TRIBUNAL DE JUICIO ORAL EN LO PENAL ARICA,Ordinaria,TRAFICO DE PEQUEÑAS CANTIDADES (ART. 4).,2018-01-10,2018-03-14,mar-18,2018.0,SENTENCIA,63.0,1.0
4,4.0,10.0,C.A. DE ARICA,988,TRIBUNAL DE JUICIO ORAL EN LO PENAL ARICA,102-2018,Ordinaria,812,RECEPTACION. ART. 456 BIS A.,2018-04-02,abr-18,2018.0,1.0,4.0,10.0,C.A. DE ARICA,TRIBUNAL DE JUICIO ORAL EN LO PENAL ARICA,Ordinaria,RECEPTACION. ART. 456 BIS A.,2018-04-02,2018-05-29,may-18,2018.0,SENTENCIA,57.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3653831,NaN,NaN,NaN,1323,NaN,247-2015,NaN,303,NaN,NaT,NaN,NaN,NaN,721435.0,91.0,C.A. DE SAN MIGUEL,TRIBUNAL DE JUICIO ORAL EN LO PENAL SAN BERNARDO,Ordinaria,FALSIFICACION O USO MALICIOSO DE DOCUMENTOS PR...,2015-12-09,2016-03-24,mar-16,2016.0,SENTENCIA,106.0,1.0
3653832,NaN,NaN,NaN,1323,NaN,251-2015,NaN,7037,NaN,NaT,NaN,NaN,NaN,721446.0,91.0,C.A. DE SAN MIGUEL,TRIBUNAL DE JUICIO ORAL EN LO PENAL SAN BERNARDO,Ordinaria,TRAFICO DE PEQUEÑAS CANTIDADES (ART. 4).,2015-12-10,2016-01-30,ene-16,2016.0,SENTENCIA,51.0,1.0
3653833,NaN,NaN,NaN,1323,NaN,251-2015,NaN,10009,NaN,NaT,NaN,NaN,NaN,721448.0,91.0,C.A. DE SAN MIGUEL,TRIBUNAL DE JUICIO ORAL EN LO PENAL SAN BERNARDO,Ordinaria,POSESION O TENENCIA DE ARMAS PROHIBIDAS,2015-12-10,2016-01-30,ene-16,2016.0,SENTENCIA,51.0,1.0
3653834,NaN,NaN,NaN,1323,NaN,251-2015,NaN,10099,NaN,NaT,NaN,NaN,NaN,721449.0,91.0,C.A. DE SAN MIGUEL,TRIBUNAL DE JUICIO ORAL EN LO PENAL SAN BERNARDO,Ordinaria,OTROS DELITOS DE LA LEY DE CONTROL DE ARMAS (L...,2015-12-10,2016-01-30,ene-16,2016.0,SENTENCIA,51.0,1.0


In [10]:
df_fulldata_materia.columns

Index(['index_x', 'COD. CORTE_x', 'CORTE_x', 'COD. TRIBUNAL', 'TRIBUNAL_x',
       'RIT', 'TIPO CAUSA_x', 'COD. MATERIA', 'MATERIA_x', 'FECHA INGRESO_x',
       'MES INGRESO', 'AÑO INGRESO', 'TOTAL INGRESOS POR MATERIAS', 'index_y',
       'COD. CORTE_y', 'CORTE_y', 'TRIBUNAL_y', 'TIPO CAUSA_y', 'MATERIA_y',
       'FECHA INGRESO_y', 'FECHA TERMINO', 'MES TERMINO', 'AÑO TERMINO',
       'MOTIVO TERMINO', 'DURACION CAUSA', 'TOTAL TERMINOS'],
      dtype='object')

In [11]:
columnas_drop = ['index_x', 'index_y', 'MES INGRESO', 'MES TERMINO']

df_fulldata_materia.drop(columnas_drop, axis = 'columns', inplace = True)

Aca podemos completar algunos datos faltantes utilizando los datos del dataset INGRESOS:

    - FECHA INGRESO_x en Termino
    - MATERIA_x
o

    - FECHA INGRESO_y en Ingreso
    - MATERIA_y
    
Esto debido a que los datos faltantes se deben principalmente a formas de tramitar las causas en los diferentes tribunales. Esto despues de analizar una cantidad importante de Causas del Tribunal Oral en lo penal de Punta Arenas , cod: Tribunal 991 y junto a Maritza Romero, Jefa de Causas y abogada donde se concluyo:

"Bien. Llevo 5 causas revisadas y sin perjuicio que desentrañaré las otras (3), puedo decir a este altura, que todos los delitos (materias) que no tienen término, obedecen a situaciones a las que el siagj no da respuesta, y dependiendo de los criterios de registro de la tramitación, dan origen a esta falta de término.
 52-2018 , uno de los delitos de la acusación se subsumió (incluyó, por decirlo de algún modo) en el otro, por tanto, el delito subsumido no llevó pena. En esa época, el criterio era dar cuenta de todos los delitos de la acusación, y registrar qué pasó con ellos, y para este caso no había cómo registrar y queda sin término. Actualmente simplemente los eliminamos y ya no damos cuenta de los delitos de la acusación versus sentencia, sino solo acusación.
 111-2017 No hay término para un  delito respecto del cual el tribunal se declaró incompetente.
 155-2017 , caso parecido al primero en el efecto, aunque no en lo jurídico. Acá hubo un concurso aparente de leyes penales, es decir una hecho que puede encuadrarse en dos delitos, y se acusó por los dos , se resolvió prefiriendo uno de ellos, el otro queda sin término."

In [12]:
df_fulldata_materia = df_fulldata_materia.progress_apply(transformdata.faltantes_materia, axis=1)

100%|██████████| 3653836/3653836 [07:50<00:00, 7770.11it/s] 


In [13]:
columnas_drop = ['TIPO CAUSA_y', 'MATERIA_y', 'TRIBUNAL_y', 'COD. CORTE_y', 'CORTE_y', 'FECHA INGRESO_y']

df_fulldata_materia.drop(columnas_drop, axis = 'columns', inplace = True)

In [14]:
df_fulldata_materia.rename(columns = {'COD. CORTE_x':'COD. CORTE',
                                      'CORTE_x':'CORTE',
                                      'TRIBUNAL_x':'TRIBUNAL',
                                      'TIPO CAUSA_x':'TIPO CAUSA',
                                      'MATERIA_x':'MATERIA',
                                      'FECHA INGRESO_x':'FECHA INGRESO'
                                     }, inplace = True)

In [15]:
df_fulldata_materia.sample(20)

,COD. CORTE,CORTE,COD. TRIBUNAL,TRIBUNAL,RIT,TIPO CAUSA,COD. MATERIA,MATERIA,FECHA INGRESO,AÑO INGRESO,TOTAL INGRESOS POR MATERIAS,FECHA TERMINO,AÑO TERMINO,MOTIVO TERMINO,DURACION CAUSA,TOTAL TERMINOS
2007373,91.0,C.A. DE SAN MIGUEL,1230,UNDECIMO JUZGADO DE GARANTIA SANTIAGO,7696-2017,Ordinaria,812,RECEPTACION. ART. 456 BIS A.,2017-09-02,2017.0,1.0,2018-09-03,2018.0,DECLARA SOBRESEIMIENTO DEFINITIVO,366.0,1.0
2219289,30.0,C.A. DE VALPARAISO,1046,JUZGADO DE GARANTIA VIÑA DEL MAR,9943-2016,Ordinaria,204,VIOLACION DE MORADA.,2016-10-18,2016.0,1.0,2017-05-22,2017.0,DECLARA INCOMPETENCIA,216.0,1.0
1384009,91.0,C.A. DE SAN MIGUEL,1240,JUZGADO DE GARANTIA TALAGANTE,625-2019,Ordinaria,816,ESTAFAS Y OTRAS DEFRAUDACIONES CONTRA PARTICUL...,2019-02-01,2019.0,1.0,2019-02-04,2019.0,COMUNICA Y/O APLICA DECISION PPIO DE OPORTUNIDAD,3.0,1.0
2701618,91.0,C.A. DE SAN MIGUEL,1229,DECIMO JUZGADO DE GARANTIA SANTIAGO,4150-2016,Ordinaria,518,PORTE DE ARMA CORTANTE O PUNZANTE (288 BIS).,2016-11-16,2016.0,1.0,2016-11-16,2016.0,COMUNICA Y/O APLICA DECISION PPIO DE OPORTUNIDAD,0.0,1.0
1021854,46.0,C.A. DE CONCEPCION,1081,JUZGADO DE GARANTIA CHIGUAYANTE,1260-2019,Ordinaria,13036,LESIONES LEVES.,2019-06-24,2019.0,1.0,2019-12-27,2019.0,DECLARA SOBRESEIMIENTO DEFINITIVO,186.0,1.0
7862,10.0,C.A. DE ARICA,992,JUZGADO DE GARANTIA ARICA,6406-2018,Ordinaria,710,LESIONES MENOS GRAVES.,2018-08-28,2018.0,1.0,2018-10-05,2018.0,SENTENCIA,38.0,1.0
3566468,55.0,C.A. DE VALDIVIA,1084,JUZGADO DE GARANTIA VALDIVIA,3647-2019,Ordinaria,816,ESTAFAS Y OTRAS DEFRAUDACIONES CONTRA PARTICUL...,2019-06-21,2019.0,NaN,2019-06-21,2019.0,DECLARA SOBRESEIMIENTO DEFINITIVO,0.0,1.0
291401,46.0,C.A. DE CONCEPCION,159,JUZGADO DE LETRAS Y GARANTIA LAJA,199-2018,Ordinaria,524,AMENAZAS SIMPLES CONTRA PERSONAS Y PROPIEDADES...,2018-04-11,2018.0,1.0,2018-04-11,2018.0,COMUNICA Y/O APLICA DECISION PPIO DE OPORTUNIDAD,0.0,1.0
83537,25.0,C.A. DE LA SERENA,929,JUZGADO DE GARANTIA LA SERENA,2813-2018,Ordinaria,848,HURTO SIMPLE POR UN VALOR DE MEDIA A MENOS DE ...,2018-04-27,2018.0,1.0,2018-05-15,2018.0,DECLARA INCOMPETENCIA,18.0,1.0
1563779,30.0,C.A. DE VALPARAISO,1057,JUZGADO DE GARANTIA QUILLOTA,2578-2017,Ordinaria,12078,CONDUC. BAJO INFLUEN DEL ALCOHOL CON O SIN DAÑ...,2017-11-16,2017.0,1.0,2018-12-07,2018.0,DECLARA SOBRESEIMIENTO DEFINITIVO,386.0,1.0


In [16]:
df_fulldata_materia['AÑO INGRESO'].value_counts()

2016.0    742682
2015.0    742424
2018.0    738755
2017.0    735229
2019.0    694746
Name: AÑO INGRESO, dtype: int64

In [17]:
print(f"Existen un total de {len(df_fulldata_materia)} causas por materias")

Existen un total de 3653836 causas por materias


De los cuales :

In [18]:
df_fulldata_materia['MOTIVO TERMINO'].value_counts()

DECLARA SOBRESEIMIENTO DEFINITIVO                   755204
APROBACION NO INICIO INVESTIGACION                  639480
SENTENCIA                                           593329
SIN TERMINO                                         510301
COMUNICA Y/O APLICA DECISION PPIO DE OPORTUNIDAD    367695
ACOGE REQUERIMIENTO (MONITORIO)                     317926
NO PERSEVERAR EN EL PROCEDIMIENTO                   211676
DECLARA INCOMPETENCIA                                99943
CERTIFICA CUMPLIMIENTO ART 468                       64963
ACUMULACION                                          53996
DECLARA INCOMPETENCIA RPA                            23456
DECLARA INADMISIBILIDAD DE LA QUERELLA                9900
ABANDONO DE LA QUERELLA                               5967
Name: MOTIVO TERMINO, dtype: int64

Acá analizamos si existe alguna relación entre Total Ingresos por Matería y Total Terminos.

-----------------------------------------------------------------

In [19]:
filtro_oral = df_fulldata_materia[df_fulldata_materia['TRIBUNAL'].str.contains('ORAL')]
filtro_garantia = df_fulldata_materia[df_fulldata_materia['TRIBUNAL'].str.contains('GARANTIA')]

In [20]:
filtro_oral.sample(10)

,COD. CORTE,CORTE,COD. TRIBUNAL,TRIBUNAL,RIT,TIPO CAUSA,COD. MATERIA,MATERIA,FECHA INGRESO,AÑO INGRESO,TOTAL INGRESOS POR MATERIAS,FECHA TERMINO,AÑO TERMINO,MOTIVO TERMINO,DURACION CAUSA,TOTAL TERMINOS
3650360,90.0,C.A. DE SANTIAGO,1245,SEGUNDO TRIBUNAL DE JUICIO ORAL EN LO PENAL SA...,606-2015,Ordinaria,518,PORTE DE ARMA CORTANTE O PUNZANTE (288 BIS).,2015-12-03,2015.0,NaN,2016-03-11,2016.0,SENTENCIA,99.0,1.0
2696342,90.0,C.A. DE SANTIAGO,1321,TRIBUNAL DE JUICIO ORAL EN LO PENAL COLINA,35-2016,Ordinaria,10001,"POSESION, TENENCIA O PORTE DE ARMAS SUJETAS A ...",2016-02-24,2016.0,1.0,2016-09-14,2016.0,SENTENCIA,203.0,1.0
2692929,90.0,C.A. DE SANTIAGO,1245,SEGUNDO TRIBUNAL DE JUICIO ORAL EN LO PENAL SA...,138-2016,Ordinaria,637,VIOLACION DE MAYOR DE 14 AÑOS.,2016-03-14,2016.0,1.0,2016-05-06,2016.0,SENTENCIA,53.0,1.0
415634,55.0,C.A. DE VALDIVIA,1095,TRIBUNAL DE JUICIO ORAL EN LO PENAL VALDIVIA,8-2018,Ordinaria,702,HOMICIDIO.,2018-01-12,2018.0,1.0,2018-03-27,2018.0,SENTENCIA,74.0,1.0
847829,30.0,C.A. DE VALPARAISO,1048,TRIBUNAL DE JUICIO ORAL EN LO PENAL VIÑA DEL MAR,394-2019,Ordinaria,14005,NEGATIVA A EFECTUARSE EXAMEN. ART. 195 BIS LEY...,2019-08-29,2019.0,1.0,NaT,NaN,SIN TERMINO,NaN,NaN
924338,35.0,C.A. DE RANCAGUA,1071,TRIBUNAL DE JUICIO ORAL EN LO PENAL RANCAGUA,485-2019,Ordinaria,7037,TRAFICO DE PEQUEÑAS CANTIDADES (ART. 4).,2019-10-02,2019.0,1.0,2019-11-25,2019.0,SENTENCIA,54.0,1.0
3640732,46.0,C.A. DE CONCEPCION,1326,TRIBUNAL DE JUICIO ORAL EN LO PENAL CAÑETE,190-2015,Ordinaria,7038,CONSUMO/PORTE EN LUG.PUB.O PRIV.C/PREVIO CONCI...,2015-12-09,2015.0,NaN,2016-09-16,2016.0,SENTENCIA,282.0,1.0
875894,30.0,C.A. DE VALPARAISO,1059,TRIBUNAL DE JUICIO ORAL EN LO PENAL SAN FELIPE,41-2019,Ordinaria,710,LESIONES MENOS GRAVES.,2019-04-17,2019.0,1.0,2019-07-30,2019.0,SENTENCIA,104.0,1.0
3403889,90.0,C.A. DE SANTIAGO,1246,TERCER TRIBUNAL DE JUICIO ORAL EN LO PENAL SAN...,155-2015,Ordinaria,809,ROBO EN LUGAR HABITADO O DESTINADO A LA HABITA...,2015-07-03,2015.0,1.0,2015-08-05,2015.0,SENTENCIA,33.0,1.0
168611,30.0,C.A. DE VALPARAISO,1053,TRIBUNAL DE JUICIO ORAL EN LO PENAL QUILLOTA,41-2018,Ordinaria,702,HOMICIDIO.,2018-03-27,2018.0,1.0,2018-05-29,2018.0,SENTENCIA,63.0,1.0


In [21]:
filtro_garantia.sample(10)

,COD. CORTE,CORTE,COD. TRIBUNAL,TRIBUNAL,RIT,TIPO CAUSA,COD. MATERIA,MATERIA,FECHA INGRESO,AÑO INGRESO,TOTAL INGRESOS POR MATERIAS,FECHA TERMINO,AÑO TERMINO,MOTIVO TERMINO,DURACION CAUSA,TOTAL TERMINOS
1295188,90.0,C.A. DE SANTIAGO,1235,JUZGADO DE GARANTIA COLINA,3073-2019,Ordinaria,7038,CONSUMO/PORTE EN LUG.PUB.O PRIV.C/PREVIO CONCI...,2019-05-28,2019.0,1.0,2019-05-28,2019.0,ACOGE REQUERIMIENTO (MONITORIO),0.0,1.0
406860,55.0,C.A. DE VALDIVIA,1087,JUZGADO DE GARANTIA RIO NEGRO,719-2018,Ordinaria,12077,NO DAR CUENTA DE ACCIDENTE DE TRANSITOART. 195...,2018-08-21,2018.0,1.0,NaT,NaN,SIN TERMINO,NaN,NaN
3020841,40.0,C.A. DE TALCA,961,JUZGADO DE GARANTIA CURICO,2248-2015,Ordinaria,7006,CULTIVO/COSECHA ESPEC.VEGETALES PRODUCTORAS ES...,2015-04-24,2015.0,1.0,2015-04-24,2015.0,ACOGE REQUERIMIENTO (MONITORIO),0.0,1.0
3234357,90.0,C.A. DE SANTIAGO,1220,PRIMER JUZGADO DE GARANTIA SANTIAGO,415-2015,Ordinaria,524,AMENAZAS SIMPLES CONTRA PERSONAS Y PROPIEDADES...,2015-02-05,2015.0,1.0,2015-04-20,2015.0,NO PERSEVERAR EN EL PROCEDIMIENTO,74.0,1.0
2410274,46.0,C.A. DE CONCEPCION,1082,JUZGADO DE GARANTIA CONCEPCION,9840-2016,Ordinaria,524,AMENAZAS SIMPLES CONTRA PERSONAS Y PROPIEDADES...,2016-10-17,2016.0,1.0,2016-11-02,2016.0,DECLARA SOBRESEIMIENTO DEFINITIVO,16.0,1.0
2013208,91.0,C.A. DE SAN MIGUEL,1231,DUODECIMO JUZGADO DE GARANTIA SANTIAGO,1260-2017,Ordinaria,1001,PRESUNTA DESGRACIA.,2017-03-15,2017.0,1.0,2017-03-15,2017.0,APROBACION NO INICIO INVESTIGACION,0.0,1.0
1875160,90.0,C.A. DE SANTIAGO,1223,CUARTO JUZGADO DE GARANTIA SANTIAGO,4668-2017,Ordinaria,13028,HURTO FALTA (494 BIS CODIGO PENAL).,2017-05-25,2017.0,1.0,2017-12-06,2017.0,DECLARA SOBRESEIMIENTO DEFINITIVO,195.0,1.0
594756,90.0,C.A. DE SANTIAGO,1235,JUZGADO DE GARANTIA COLINA,109-2018,Ordinaria,1001,PRESUNTA DESGRACIA.,2018-01-08,2018.0,1.0,2018-01-08,2018.0,APROBACION NO INICIO INVESTIGACION,0.0,1.0
1324180,91.0,C.A. DE SAN MIGUEL,1231,DUODECIMO JUZGADO DE GARANTIA SANTIAGO,4712-2019,Ordinaria,710,LESIONES MENOS GRAVES.,2019-10-16,2019.0,1.0,2019-10-16,2019.0,APROBACION NO INICIO INVESTIGACION,0.0,1.0
2645570,90.0,C.A. DE SANTIAGO,1228,NOVENO JUZGADO DE GARANTIA SANTIAGO,2866-2016,Ordinaria,7006,CULTIVO/COSECHA ESPEC.VEGETALES PRODUCTORAS ES...,2016-03-18,2016.0,1.0,2016-04-19,2016.0,DECLARA SOBRESEIMIENTO DEFINITIVO,32.0,1.0


In [22]:
filtro_oral['TRIBUNAL'].value_counts()

SEXTO TRIBUNAL DE JUICIO ORAL EN LO PENAL SANTIAGO      5197
TRIBUNAL DE JUICIO ORAL EN LO PENAL IQUIQUE             5186
SEGUNDO TRIBUNAL DE JUICIO ORAL EN LO PENAL SANTIAGO    4762
CUARTO TRIBUNAL DE JUICIO ORAL EN LO PENAL SANTIAGO     4418
TRIBUNAL DE JUICIO ORAL EN LO PENAL RANCAGUA            4304
TRIBUNAL DE JUICIO ORAL EN LO PENAL VALPARAISO          4153
TRIBUNAL DE JUICIO ORAL EN LO PENAL CONCEPCION          4014
TRIBUNAL DE JUICIO ORAL EN LO PENAL VIÑA DEL MAR        3847
SEPTIMO TRIBUNAL DE JUICIO ORAL EN LO PENAL SANTIAGO    3328
TRIBUNAL DE JUICIO ORAL EN LO PENAL ANTOFAGASTA         3149
TRIBUNAL DE JUICIO ORAL EN LO PENAL SAN BERNARDO        3116
TRIBUNAL DE JUICIO ORAL EN LO PENAL ARICA               3024
TERCER TRIBUNAL DE JUICIO ORAL EN LO PENAL SANTIAGO     2943
TRIBUNAL DE JUICIO ORAL EN LO PENAL TALCA               2462
TRIBUNAL DE JUICIO ORAL EN LO PENAL LA SERENA           2340
PRIMER TRIBUNAL DE JUICIO ORAL EN LO PENAL SANTIAGO     2171
TRIBUNAL DE JUICIO ORAL 

In [23]:
filtro_oral['MATERIA'].value_counts()

TRAFICO DE PEQUEÑAS CANTIDADES (ART. 4).                        8279
RECEPTACION. ART. 456 BIS A.                                    5851
TRAFICO ILICITO DE DROGAS (ART. 3).                             5770
ROBO CON INTIMIDACION.                                          5595
ROBO EN LUGAR HABITADO O DESTINADO A LA HABITACION.             4504
                                                                ... 
AMPLIACION DE PARTE.                                               1
LAVADO DE DINERO.                                                  1
CRIMENES Y SIMPLES DELITOS SEGURIDAD INTERIOR DEL ESTADO.          1
EXTRANJE. INGRESAN O INTENTAN EGRESAR C/DOCUMENTOS FALSIFIC.       1
AMENAZAS SIMPLES CONTRA PERSONAS Y PROPIEDADES ART. 296 Nº3        1
Name: MATERIA, Length: 358, dtype: int64

In [24]:
filtro_oral['MOTIVO TERMINO'].value_counts()

SENTENCIA                            83183
SIN TERMINO                           9191
DECLARA SOBRESEIMIENTO DEFINITIVO     2085
ACUMULACION                            781
DECLARA INCOMPETENCIA                  249
ABANDONO DE LA QUERELLA                 54
DECLARA INCOMPETENCIA RPA                3
Name: MOTIVO TERMINO, dtype: int64

## CARGA DATA A ARCHIVOS FUTHER

In [25]:
# Directorio donde se guardaran archivos feather
df_fulldata_materia.reset_index(inplace = True)
filtro_oral.reset_index(inplace = True)
filtro_garantia.reset_index(inplace = True)
os.makedirs(path_processed, exist_ok = True)  

# Guardamos dataset como archivo feather

df_fulldata_materia.to_feather(f'{path_processed}/consolidated_Materia.feather')
filtro_oral.to_feather(f'{path_processed}/consolidated_JuicioOralesMateria.feather')
filtro_garantia.to_feather(f'{path_processed}/consolidated_CausasGarantiaMateria.feather')